In [336]:
import numpy as np
import matplotlib.pyplot as plt
from math import exp


In [377]:
def objective1(x):
    dim = len(x)
    return 1 - exp(-sum([(x[i] - (1 / (dim**0.5)))**2 for i in range(dim)]))


def objective2(x):
    dim = len(x)
    return 1 - exp(-sum([(x[i] + (1 / (dim**0.5)))**2 for i in range(dim)]))

# Параметры генетического алгоритма
crossover_rate = 0.7
crossover_parametr = 3
mutation_rate = 0.01
mutation_parametr = 4

population_size = 100
dimensions = 3
generations = 15


# Диапазон значений переменных
x_min, x_max = -4, 4


In [378]:
def initialize_population(size, dimensions):
    return list(np.random.rand(size, dimensions))

def select_sorted_items(population, num_to_select):
    selected_indices = np.random.choice(range(len(population)), size=num_to_select*2 if len(population) >= num_to_select*2 else len(population), replace=False)
    selected = np.array(population, dtype=object)[selected_indices]

    fronts = FNDS(selected)
    selected = []
    for front in fronts:
        if len(front) == 0:
            continue
        dists = CDA(front)
        selected.extend([np.array(j[0]) for j in sorted(dists.items(), key=lambda x:x[1], reverse=True)])
        if len(selected) >= num_to_select:
            break
    return selected[:num_to_select]

def get_beta(crossover_parametr):
    u = np.random.rand()
    if u <= 0.5:
        return (2 * u) ** (1 / (crossover_parametr + 1))
    else:
        return (1 / (2 * (1 - u))) ** (1 / (crossover_parametr + 1))

def get_delta(mutation_parametr):
    r = np.random.rand()
    if r < 0.5:
        return (2 * r) ** (1 / (mutation_parametr + 1)) - 1
    else:
        return 1 - (2 * (1 - r)) ** (1 / (mutation_parametr + 1))

def crossover(parent1, parent2, crossover_parametr=np.random.randint(1, 100)):
    child1, child2 = [], []
    for i in range(len(parent1)):
        beta = get_beta(crossover_parametr)
        child1.append(0.5 * ((1 + beta) * parent1[i] + (1 - beta) * parent2[i]))
        if child1[i] > x_max:
            child1[i] = x_max
        elif child1[i] < x_min:
            child1[i] = x_min
        child2.append(0.5 * ((1 - beta) * parent1[i] + (1 + beta) * parent2[i]))
        if child2[i] > x_max:
            child2[i] = x_max
        elif child2[i] < x_min:
            child2[i] = x_min
    return child1, child2

def mutate(cleen, mutation_parametr=np.random.randint(1, 100)):
    mutant = []
    for i in range(len(cleen)):
        if np.random.rand() < 1 / len(parent1):
            delta = get_delta(mutation_parametr)
            mutant.append(cleen[i] + delta * (x_max - x_min))
            if mutant[i] > x_max:
                mutant[i] = x_max
            elif mutant[i] < x_min:
                mutant[i] = x_min
        else:
            mutant.append(cleen[i])
    return mutant


In [379]:
def CDA(population):
    l = len(population)
    dist = dict()
    for i in range(l):
        dist[tuple(population[i])] = 0
    for func in [objective1, objective2]:
        new_population = sorted(population, key=func)

        dist[tuple(new_population[0])] = 10e6
        dist[tuple(new_population[-1])] = 10e6
        scale = func(new_population[-1]) - func(new_population[0])
        if not scale:
            scale = 1
        for i in range(1, l - 1):
            dist[tuple(new_population[i])] += (func(new_population[i + 1]) - func(new_population[i - 1])) / scale
    return dist

def dominates(individual1, individual2):
    and_condition = True
    or_condition = False
    for func in [objective1, objective2]:
        first, second =  func(individual1), func(individual2)
        and_condition = and_condition and first <= second
        or_condition = or_condition or first < second
    return (and_condition and or_condition)

def FNDS(population):

    fronts = [[]]
    domination_count = dict()
    dominated_solutions = dict()
    rank = dict()
    for individual in population:
        domination_count[tuple(individual)] = 0
        dominated_solutions[tuple(individual)] = []
        for other_individual in population:
            if dominates(individual, other_individual):
                dominated_solutions[tuple(individual)].append(other_individual)
            elif dominates(other_individual, individual):
                domination_count[tuple(individual)] += 1
        if domination_count[tuple(individual)] == 0:
            rank[tuple(individual)] = 0
            fronts[0].append(individual)
    i = 0
    while len(fronts[i]) > 0:
        temp = []
        for individual in fronts[i]:
            for other_individual in dominated_solutions[tuple(individual)]:
                domination_count[tuple(other_individual)] -= 1
                if domination_count[tuple(other_individual)] == 0:
                    rank[tuple(other_individual)] = i + 1
                    temp.append(other_individual)
        i = i + 1
        fronts.append(temp)
    return fronts


In [380]:
parents = initialize_population(population_size, dimensions)
children = initialize_population(population_size, dimensions)
for i in range(generations):
    #     print(len(parents), len(children))
    population = parents + children
    #     print(len(population[0]))
    parents = select_sorted_items(population, population_size)
    children = []
    for j in range(0, population_size, 2):
        parent1, parent2 = select_sorted_items(parents, 2)
        child1, child2 = crossover(parent1, parent2, crossover_parametr=crossover_parametr)
        child1 = mutate(child1, mutation_parametr=mutation_parametr)
        child2 = mutate(child2, mutation_parametr=mutation_parametr)
        children.extend([child1, child2])
    print(objective1(children[0]), objective2(children[0]))

print(parents)


0.5621168659184297 0.9992382683701273
0.9999835244247451 0.9999971125436564
0.999998924907943 0.9950333190282868
0.9999999966258344 0.999939986250079
0.9818281567351937 0.9804554601386037
0.9999999992114871 0.9999994577606687
0.8927777031347837 0.4876060864487821
0.7017427307995239 0.5705881723487327
0.6292964313403755 0.9719068868101494
0.8084174881382178 0.5342252165106487
0.9999983493783845 0.999999999413799
0.9800235178333498 0.9999849652651152
0.9999703532128518 0.9902473459232499
0.9999729083042145 0.9722906212752679
0.8652587832811435 0.2989951227552905
[array([-0.49662131, -0.32889433, -0.49003444]), array([0.58026856, 0.59655969, 0.57765957]), array([-0.40888279, -0.32694343, -0.2955074 ]), array([-0.30223702, -0.3026774 , -0.28678771]), array([-0.45706603, -0.39627672, -0.29192467]), array([-0.29483223, -0.26460967, -0.25931066]), array([0.28821196, 0.15745637, 0.29978102]), array([-0.03535483,  0.03577605, -0.15823703]), array([-0.33020537, -0.18111389, -0.12468435]), array(